# 构建一个基于SQL数据的问答系统 Build a Question/Answering system over SQL data

使 LLM 系统能够查询结构化数据与非结构化文本数据在性质上可能有所不同。  
后者通常生成可在向量数据库中搜索的文本，而结构化数据的方法通常是让 LLM 编写和执行 DSL（例如 SQL）中的查询。  
在本指南中，我们将介绍在数据库中的表格数据上创建问答系统的基本方法。  
我们将介绍使用链和代理的实现。  
这些系统将允许我们询问有关数据库中数据的问题并得到自然语言答案。  
两者之间的主要区别在于，我们的代理可以根据需要多次循环查询数据库以回答问题。

## 架构
从高层次来看，这些系统的步骤如下：

 - 将问题转换为 DSL 查询：模型将用户输入转换为 SQL 查询。
 - 执行 SQL 查询：执行查询。
 - 回答问题：模型使用查询结果响应用户输入。
   
请注意，查询 CSV 中的数据可以采用类似的方法。

### Chinook数据库的脚本
https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql   
可以安装sqlite客户端执行脚本，推荐使用Navicat Premium 17

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("select * from Artist limit 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

## Chains 链
链（即 LangChain Runnable 的组合）支持步骤可预测的应用程序。  
我们可以创建一个简单的链，它接受一个问题并执行以下操作：

 - 将问题转换为 SQL 查询；
 - 执行查询；
 - 使用结果回答原始问题。
   
有些场景不受此安排的支持。例如，此系统将对任何用户输入执行 SQL 查询 - 甚至是“你好”。  
重要的是，正如我们将在下面看到的，有些问题需要多个查询才能回答。  
我们将在代理部分中解决这些情况。

### 将问题转换为 SQL 查询
SQL 链或代理中的第一步是获取用户输入并将其转换为 SQL 查询。  
LangChain 带有一个内置链：create_sql_query_chain。

In [2]:
import os
from dotenv import find_dotenv,load_dotenv

_ = load_dotenv(find_dotenv())

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4",
    api_key=os.environ["ZHIPUAI_API_KEY"],
    model="glm-4",
)

In [30]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm,db)
resp = chain.invoke({"question":"有多少名员工?"})
resp

'SELECT COUNT("EmployeeId") FROM "Employee"\nSQLResult:'

In [31]:
db.run(resp.split("\n")[0])

'[(8,)]'

In [32]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### 执行 SQL 查询
现在我们已经生成了 SQL 查询，我们将要执行它。这是创建 SQL 链中最危险的部分。  
请仔细考虑是否可以对数据运行自动查询。尽可能减少数据库连接权限。  
考虑在查询执行之前向链中添加人工批准步骤。

我们可以使用 QuerySQLDatabaseTool 轻松地将查询执行添加到我们的链中：

In [33]:
from operator import itemgetter
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnablePassthrough

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm,db)
chain = RunnablePassthrough.assign(query=lambda x: write_query.invoke(x).split("\n")[0]) | execute_query
chain.invoke({"question":"有多少名员工?"})

'[(8,)]'

### 回答问题
现在我们已经有了自动生成和执行查询的方法，我们只需要将原始问题和 SQL 查询结果结合起来即可生成最终答案。  
我们可以通过再次将问题和结果传递给 LLM 来实现这一点：

In [34]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """给出以下用户问题、相应的 SQL 查询和 SQL 结果，回答用户问题。

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=lambda x: write_query.invoke(x).split("\n")[0]).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "有多少名员工?"})

'根据提供的SQL查询结果，有8名员工。'

让我们回顾一下上述 LCEL 中发生的事情。假设调用了此链。

 - 在第一个 RunnablePassthrough.assign 之后，我们有一个包含两个元素的可运行程序：
{"question": question, "query": write_query.invoke(question)}
其中 write_query 将生成一个 SQL 查询来回答问题。
 - 在第二个 RunnablePassthrough.assign 之后，我们添加了第三个元素“result”，其中包含 execute_query.invoke(query)，其中 query 是在上一步中计算的。
 - 这三个输入被格式化为提示并传递到 LLM。
 - StrOutputParser() 提取输出消息的字符串内容。  
请注意，我们正在将 LLM、工具、提示和其他链组合在一起，但由于每个链都实现了 Runnable 接口，因此它们的输入和输出可以以合理的方式绑定在一起。

## 代理
LangChain 有一个 SQL 代理，它提供了一种比链更灵活的与 SQL 数据库交互的方式。使用 SQL 代理的主要优点是：

 - 它可以根据数据库的架构以及数据库的内容（如描述特定表）回答问题。
 - 它可以通过运行生成的查询、捕获回溯并正确地重新生成它来从错误中恢复。
 - 它可以根据需要多次查询数据库以回答用户问题。
 - 它将仅通过从相关表中检索架构来保存令牌。
   
为了初始化代理，我们将使用 SQLDatabaseToolkit 创建一组工具：

 - 创建和执行查询
 - 检查查询语法
 - 检索表描述
 - ... 等等

In [35]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db,llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002783ED7ABD0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002783ED7ABD0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002783ED7ABD0>),
 QuerySQLCheckerTool(description='Use this tool to 

### System Prompt
我们还想为我们的代理创建一个系统提示。这将包括如何操作的说明。

In [36]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """您是设计用于与 SQL 数据库交互的代理。
给定一个输入问题，创建一个语法正确的 SQLite 查询来运行，然后查看查询结果并返回答案。
除非用户指定他们希望获得的特定示例数量，否则请始终将查询限制为最多 5 个结果。
您可以按相关列对结果进行排序，以返回数据库中最有趣的示例。
切勿查询特定表中的所有列，只询问给定问题的相关列。
您可以使用与数据库交互的工具。
仅使用以下工具。仅使用以下工具返回的信息来构建最终答案。
在执行查询之前，您必须仔细检查查询。如果在执行查询时出现错误，请重写查询并重试。

不要对数据库执行任何 DML 语句（INSERT、UPDATE、DELETE、DROP 等）。

首先，您应该始终查看数据库中的表以查看可以查询的内容。
不要跳过此步骤。
然后您应该查询最相关的表的模式。"""

system_message = SystemMessage(content=SQL_PREFIX)

In [37]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

In [38]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="哪个国家的顾客花费最多?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8775526313750640562', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 728, 'total_tokens': 741}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-63a92921-8a96-4df1-aaab-3126da4b3827-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_8775526313750640562'}])]}}
----
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', tool_call_id='call_8775526313750640562')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8775532807741551406', 'function': {'arguments': '{"table_names":"Customer, Invoice"}', 'name': 'sql_db_schema'},

## 处理高基数列
为了过滤包含专有名词（例如地址、歌曲名称或艺术家）的列，我们首先需要仔细检查拼写，以便正确过滤数据。

我们可以通过创建一个向量存储来实现这一点，该向量存储包含数据库中存在的所有不同专有名词。  
然后，我们可以让代理在每次用户在问题中包含专有名词时查询该向量存储，以找到该单词的正确拼写。  
通过这种方式，代理可以在构建目标查询之前确保它了解用户指的是哪个实体。

首先，我们需要每个实体的唯一值，为此我们定义一个函数将结果解析为元素列表：

In [39]:
import ast
import re

def query_as_list(db,query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b","",string).strip() for string in res]
    return list(set(res))

artists = query_as_list(db, "SELECT Name FROM Artist")
print(artists[:5])
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

['UB40', 'Sabotage E Instituto', "Guns N' Roses", 'JET', 'Planet Hemp']


['B-Sides -',
 'St. Anger',
 'The Real Thing',
 'Sir Neville Marriner: A Celebration',
 'International Superhits']

利用这个函数，我们可以创建一个代理可以自行执行的检索工具。

In [45]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import BaichuanTextEmbeddings


embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

vector_db = FAISS.from_texts(artists[:5] + albums[:5], embeddings)
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """使用查找要过滤的值。输入是专有名词的近似拼写，输出是 \
有效专有名词。使用与搜索最相似的名词。"""
retriever_tool = create_retriever_tool(
    retriever,
    name="搜索专有名词",
    description=description,
)

In [50]:
print(retriever_tool.invoke("Guns N' Roses"))

Guns N' Roses

B-Sides -

St. Anger

The Real Thing

Sir Neville Marriner: A Celebration


这样，如果代理确定需要根据“爱丽丝链”之类的艺术家编写过滤器，它可以首先使用检索工具来观察列的相关值。

将它们放在一起：

In [47]:
llm = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
)

In [48]:
system = """您是设计用于与 SQL 数据库交互的代理。
给定一个输入问题，创建一个语法正确的 SQLite 查询来运行，然后查看查询结果并返回答案。
除非用户指定他们希望获得的特定示例数量，否则请始终将查询限制为最多 5 个结果。
您可以按相关列对结果进行排序，以返回数据库中最有趣的示例。
切勿查询特定表中的所有列，仅询问给定问题的相关列。
您可以使用与数据库交互的工具。
仅使用给定的工具。仅使用工具返回的信息来构建最终答案。
在执行查询之前，您必须仔细检查查询。如果在执行查询时出现错误，请重写查询并重试。

请勿对数据库执行任何 DML 语句（INSERT、UPDATE、DELETE、DROP 等）。

您可以访问以下表格：{table_names}

如果您需要过滤专有名词，则必须始终先使用“search_proper_nouns”工具查找过滤值！
不要试图猜测专有名词 - 使用此功能查找类似名称。""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, messages_modifier=system_message)

In [52]:
for s in agent.stream(
    {"messages": [HumanMessage(content="How many albums does Guns N' Roses have?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'f9f9016NGeWoJtV', 'function': {'arguments': '{"query": "Guns N\' Roses"}', 'name': '搜索专有名词'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 790, 'total_tokens': 809}, 'model_name': 'Baichuan4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6ce97472-766d-471d-8550-9cd2031cbd4e-0', tool_calls=[{'name': '搜索专有名词', 'args': {'query': "Guns N' Roses"}, 'id': 'f9f9016NGeWoJtV'}])]}}
----
{'tools': {'messages': [ToolMessage(content="Guns N' Roses\n\nB-Sides -\n\nSt. Anger\n\nThe Real Thing\n\nSir Neville Marriner: A Celebration", name='搜索专有名词', tool_call_id='f9f9016NGeWoJtV')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '193c016NGeaosjx', 'function': {'arguments': '{"query": "SELECT COUNT(*) FROM Album WHERE Title = \'B-Sides -\' OR Title = \'St. Anger\' OR Title = \